In [ ]:
import pandas as pd
from mlp_ensemble import MlpEnsemble

#### **Getting training && validation data**

In [ ]:
df_train = pd.read_csv('../databases/training.csv')
X_train, y_train = df_train.drop(['IND_BOM_1_1'], axis=1), df_train['IND_BOM_1_1']

In [ ]:
df_valid = pd.read_csv('../databases/validation.csv')
X_val, y_val = df_valid.drop(['IND_BOM_1_1'], axis=1), df_valid['IND_BOM_1_1']

#### **MLP Ensemble**

In [ ]:
models_params = [
    {
        'hidden_layer_units': 32,
        'hidden_layers': 1,
        'learning_rate': 0.01,
        'max_iter': 5000,
        'batch_size': 128,
        'activation': 'sigmoid',
        'optimizer': 'Adam',
        'output_activation': 'sigmoid',
        'loss_function': 'mse',
        'dropout_rate': 0.5
    },
    {
        'hidden_layer_units': 32,
        'hidden_layers': 1,
        'learning_rate': 0.01,
        'max_iter': 10000,
        'batch_size': 128,
        'activation': 'sigmoid',
        'optimizer': 'Adam',
        'output_activation': 'sigmoid',
        'loss_function': 'mse',
        'dropout_rate': 0.2
    }
]

In [ ]:
input_dimension = X_train.shape[1]

ensemble_model = MlpEnsemble(models_params, input_dimension)

In [ ]:
ensemble_model.fit(X_train, y_train, X_val, y_val)

In [ ]:
ensemble_model.save_model()